In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import time

In [2]:
df = pd.read_csv(r"C:\Users\anpar\Python\Pandas\Classification_train.csv")

In [3]:
X = np.array(df.iloc[:, 1:])
y = np.array(df.iloc[:, 0])

In [4]:
class Standarize:
    
    mean = std = None
    
    def fit(self, X):
        self.mean = X.mean(axis=0) + 1e-8
        self.std = X.std(axis=0) + 1e-8
        
        
    def fit_transform(self, X):
        self.fit(X)
        X_scaled = (X - self.mean)/self.std
        return X_scaled
        

    def transform(self, X):
        try:
            X_scaled = (X - self.mean)/self.std
            return X_scaled
        except TypeError:
            raise TypeError('No data has been provided to calculate mean and standard deviation')



In [5]:
def softmax(z):
    expZ = np.exp(z)
    return expZ/(np.sum(expZ, 0))

def relu(Z):
    A = np.maximum(0,Z)
    return A

def derivative_relu(Z):
    return np.array(Z > 0, dtype = 'float')

In [6]:
def one_hot_encoding(y):
    classes = np.unique(y)
    y_encoded = np.zeros(y.shape[0])
    
    for i in classes:
        y_df = pd.DataFrame(y).loc[y == i].copy()
        y_n = pd.DataFrame(np.zeros(y.shape[0]))
        y_n.loc[y_df.index] = 1
        y_encoded = np.c_[y_encoded, y_n]
    
    y_encoded = np.delete(y_encoded, 0, 1)
    
    return y_encoded

In [7]:
class HiddenLayer:
    def __init__(self, n_neurons=5):
        self.n_neurons = n_neurons

    def feed(self, X):
        self.X = X
        self.weights = 0.1*np.random.randn(self.n_neurons, self.X.shape[0])
        self.bias = np.random.rand(self.n_neurons, 1)
        self.out = self.weights.dot(X) + self.bias 

In [8]:
def initiate(X, neurons_each_layer):
    n_layers = len(neurons_each_layer)
    wnb_params = {}
    output = X
    
    for i in range(1, n_layers):
        layer = HiddenLayer(n_neurons=neurons_each_layer[i])
        layer.feed(output)
        output = layer.out
        wnb_params['layer'+str(i)] = layer

    return wnb_params

In [9]:
def propagate_forward(X, wnb_params):
   
    layer_feed = {}
    L = len(wnb_params)                  
    
    layer_feed['a_feed0'] = X

    for l in range(1, L):
        layer_feed['z_feed'+str(l)] = wnb_params['layer'+str(l)].weights.dot(layer_feed['a_feed'+str(l-1)]) + wnb_params['layer'+str(l)].bias
        layer_feed['a_feed'+str(l)] = relu(layer_feed['z_feed'+str(l)])
            

    layer_feed['z_feed'+str(L)] = wnb_params['layer'+str(L)].weights.dot(layer_feed['a_feed'+str(L-1)]) + wnb_params['layer'+str(L)].bias
    layer_feed['a_feed'+str(L)] = softmax(layer_feed['z_feed'+str(L)])
    
    return layer_feed['a_feed'+str(L)], layer_feed

In [10]:
def compute_cost(y, layer_feed):
    L = len(layer_feed)//2
    m = y.shape[1]
    
    cost = -(1/m)*np.sum(y*np.log(layer_feed['a_feed'+str(L)]))
        
    cost = np.squeeze(cost)
    
    return cost

In [11]:
def propagate_backward(y, wnb_params, layer_feed):
    
    grads = {}
    L = len(wnb_params)
    m = y.shape[1]
    
    grads["dl/dz"+str(L)] = layer_feed['a_feed'+str(L)] - y
    grads["dl/dw"+str(L)] = 1/m * np.dot(grads["dl/dz"+str(L)],layer_feed['a_feed'+str(L-1)].T)
    grads["dl/db"+str(L)] = 1/m * np.sum(grads["dl/dz"+str(L)], axis = 1, keepdims = True)
    
    for l in reversed(range(1, L)):
        grads["dl/dz"+str(l)] = np.dot(wnb_params['layer'+str(l+1)].weights.T,grads["dl/dz"+str(l+1)])*derivative_relu(layer_feed['a_feed'+str(l)])
            
        grads["dl/dw"+str(l)] = 1/m*np.dot(grads["dl/dz"+str(l)],layer_feed['a_feed'+str(l-1)].T)
        grads["dl/db"+str(l)] = 1/m*np.sum(grads["dl/dz"+str(l)], axis = 1, keepdims = True)

    return grads

In [12]:
def update_params(wnb_params, grads, learning_rate):

    L = len(wnb_params) 
    
    for l in range(L):
        wnb_params["layer"+str(l+1)].weights -= learning_rate * grads["dl/dw"+str(l+1)]
        wnb_params["layer"+str(l+1)].bias -= learning_rate * grads["dl/db"+str(l+1)]
        
    return wnb_params

In [13]:
def calc_accuracy(X, y, wnb_params):

    m = X.shape[1]
    y_pred, _ = propagate_forward(X, wnb_params)
    y = np.argmax(y, 0)
    y_pred = np.argmax(y_pred, 0)
    
    return np.sum((y_pred==y)/m)

In [14]:
class NeuralNetwork:
    def __init__(self, neurons_each_layer : list[int]):
        self.neurons_each_layer = neurons_each_layer

    def fit(self, X, y, learning_rate=0.1, n_iterations=1000):
        self.wnb_params = initiate(X, neurons_each_layer)
        L = len(self.wnb_params)//2
        m = y.shape[1]
        
        for i in range(0, n_iterations):
            AL, layer_feed = propagate_forward(X, self.wnb_params)
            cost = -(1/m)*np.sum(y*np.log(AL))
            grads = propagate_backward(y, self.wnb_params, layer_feed)
            self.wnb_params = update_params(self.wnb_params, grads, learning_rate)
            if i%(n_iterations/10) == 0:
                print(f'{i}th iteration done. Cost : {cost}')
        
    def predict(self,X_test):
        self.y_pred, _ = propagate_forward(X_test, self.wnb_params)
        return self.y_pred

    def check_performance(self,X_train, X_test, y_train, y_test):
        performance = {
            'Training Data' : [calc_accuracy(X_train, y_train, self.wnb_params)],
            'Testing Data' : [calc_accuracy(X_test, y_test, self.wnb_params)]
        }
        perf_df = pd.DataFrame(performance, index=['Accuracy'])
        return perf_df

In [15]:
scalar_x = Standarize()
X = scalar_x.fit_transform(X)

In [16]:
X_train, y_train = np.array(X[:int(0.9*X.shape[0]), :]).T, np.array(y[:int(0.9*y.shape[0])])
X_test, y_test = np.array(X[int(0.9*X.shape[0]):, :]).T, np.array(y[int(0.9*y.shape[0]):])

In [17]:
y_train = one_hot_encoding(y_train).T
y_test = one_hot_encoding(y_test).T

In [18]:
neurons_each_layer = [X_train.shape[0], 128, 32, y_train.shape[0]]

In [19]:
model = NeuralNetwork(neurons_each_layer=neurons_each_layer)

In [20]:
model.fit(X_train, y_train, learning_rate=0.15, n_iterations=1000)

0th iteration done. Cost : 3.4338586168296428
100th iteration done. Cost : 0.17411960203211976
200th iteration done. Cost : 0.11399889807460889
300th iteration done. Cost : 0.08826801877432844
400th iteration done. Cost : 0.07176721921925765
500th iteration done. Cost : 0.05975259866533596
600th iteration done. Cost : 0.05052150734005417
700th iteration done. Cost : 0.04311321224558635
800th iteration done. Cost : 0.03703495307251
900th iteration done. Cost : 0.03197590510673139


In [21]:
model.check_performance(X_train, X_test, y_train, y_test)

,Training Data,Testing Data
Accuracy,0.993926,0.974333


In [22]:
new_df = pd.read_csv(r"C:\Users\anpar\Python\ML_Bootcamp_Aadi\Algo\Test_Datasets\Classification_test.csv")

In [23]:
new_df[new_df.mean(axis=1) != 0]

,ID,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,15795,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,860,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5390,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11964,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11284,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,11260,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,18563,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,634,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,10057,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
X_new = new_df.iloc[:, 1:]

In [25]:
X_new

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
X_new_scaled = scalar_x.transform(X_new).T
# scalar_x.mean.shape

In [31]:
y_pred = model.predict(X_new_scaled)

C:\Users\anpar\AppData\Local\Temp\ipykernel_32304\644746286.py:2: RuntimeWarning: overflow encountered in exp
  expZ = np.exp(z)
C:\Users\anpar\AppData\Local\Temp\ipykernel_32304\644746286.py:3: RuntimeWarning: invalid value encountered in divide
  return expZ/(np.sum(expZ, 0))


In [34]:
y_new = np.argmax(y_pred, 0)

In [35]:
new_df = np.insert(new_df, 1, y_new, 1)

In [36]:
labeled_df = pd.DataFrame(new_df, columns=['ID']+['label']+['pixel'+str(i) for i in range(784)])

In [37]:
labeled_df

,ID,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,15795,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,860,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5390,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11964,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11284,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,11260,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,18563,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,634,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,10057,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
labeled_df.to_csv(r"C:\Users\anpar\Python\ML_Bootcamp_Aadi\Algo\Test_Datasets\Labeled_ANN_Classification_test.csv", index=False)